### Run Baseline: Logistic Regression with BoW Input

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import nltk

In [6]:
def load_datasets():
    """
    Load training and dev sets
    """

    train = pd.read_csv("./new_dataset/train_preprocessed.csv")
    dev = pd.read_csv("./new_dataset/val_preprocessed.csv")
    
    return train, dev

In [7]:
train, dev = load_datasets()

In [15]:
def tokenizer(text: str):
    '''
    NLTK Tweet Tokenizer -- removes handles

    @param text        string tweet
    @ret tokens        list of tokens
    '''
    text = text.lower()
    tokenizer = nltk.tokenize.TweetTokenizer(strip_handles=True)
    tokens = tokenizer.tokenize(text)
    
    return tokens

In [18]:
# Assuming we have train and test sets
Xmat_train = train["text"]
Y_train = train["Y_hard"]

Xmat_dev = dev["text"]
Y_dev = dev["Y_hard"]

# Handle missing values
Xmat_train.fillna('', inplace=True)
Xmat_dev.fillna('', inplace=True)

# Create a Bag of Words representation using the training data *only*
vectorizer = CountVectorizer(tokenizer=tokenizer)
X_train_bow = vectorizer.fit_transform(Xmat_train)
X_dev_bow = vectorizer.transform(Xmat_dev)

Run model on train and dev sets

In [23]:
# Train a Logistic Regression model
baseline_bow = LogisticRegression(max_iter=2000, multi_class='auto', solver='lbfgs')
baseline_bow.fit(X_train_bow, Y_train)

# Make predictions
Y_pred_train = baseline_bow.predict(X_train_bow)
Y_pred_dev = baseline_bow.predict(X_dev_bow)

# Evaluate the model
train_accuracy = accuracy_score(Y_train, Y_pred_train)
dev_accuracy = accuracy_score(Y_dev, Y_pred_dev)

print(f"Train accuracy: {train_accuracy}")
print(f"Dev accuracy: {dev_accuracy}")


Train accuracy: 0.9095472980379261
Dev accuracy: 0.6775


Run on test set

In [37]:
test = pd.read_csv("./new_dataset/test_preprocessed.csv")

Xmat_test = test["text"]
Y_test = test["Y_hard"]

X_test_bow = vectorizer.transform(Xmat_test)

In [38]:
Y_pred_test = baseline_bow.predict(X_test_bow)
test_accuracy = accuracy_score(Y_test, Y_pred_test)

print(f"Test accuracy: {test_accuracy}")

Test accuracy: 0.5902800390752198


In [48]:
Y_test[0]

'[0.0, 1.0, 0.0]'

In [49]:
Y_pred_test[1]

'[0.0, 1.0, 0.0]'

In [51]:
total = [0, 0, 0]
wrong = [0, 0, 0]
for y, y_pred in zip(Y_test, Y_pred_test):
    y = eval(y)
    y_pred = eval(y_pred)
    total[np.argmax(y)] += 1
    if np.argmax(y) != np.argmax(y_pred):
        # if the prediction is wrong
        wrong[np.argmax(y)] += 1


In [52]:
total

[3972, 5937, 2375]

In [58]:
percent_error = []

for i in range(len(total)):
    percent_error.append(wrong[i]/total[i])

In [59]:
percent_error

[0.5599194360523666, 0.3084049183089102, 0.41178947368421054]